In [1]:
import pandas as pd
df_spo_wiki=pd.read_csv('./data/1_spo_wiki.csv')
df_spo_wiki.info()
df_spo_wiki.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2921 entries, 0 to 2920
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subjectLabel    2921 non-null   object
 1   predicateLabel  2921 non-null   object
 2   objectLabel     2921 non-null   object
 3   subject         2921 non-null   object
 4   object          2921 non-null   object
 5   source          2921 non-null   object
dtypes: object(6)
memory usage: 137.0+ KB


,subjectLabel,predicateLabel,objectLabel,subject,object,source
0,pearl,has use,jewelry,Q43436,Q161439,Q83437
1,pearl,described by source,Brockhaus and Efron Encyclopedic Dictionary,Q43436,Q602358,Q83437
2,pearl,described by source,Encyclopædia Britannica 11th edition,Q43436,Q867541,Q83437
3,ruby,color,red,Q43088,Q3142,Q83437
4,nephrite,described by source,Brockhaus and Efron Encyclopedic Dictionary,Q138979,Q602358,Q83437


In [2]:
df=df_spo_wiki
df_pivot = pd.pivot_table(
    df.reset_index(),
    # columns=['col1'],
    index=['predicateLabel'],
    values=['index'], 
    aggfunc={
        'index': ["count"],
    },
    fill_value=0
)
df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]
df_pivot.sort_values('index_count',ascending=False).head(50)

,index_count
predicateLabel,
subclass of,901
described by source,357
instance of,257
named after,252
IMA status and/or rank,187
crystal system,158
type locality (geology),134
space group,74
country of origin,62


In [3]:
values_to_drop = {
    'described by source',
    'different from',
    "topic's main category",
    'on focus list of Wikimedia project',
    "topic's main template",
    'maintained by WikiProject'
}
df2 = df[~df['predicateLabel'].isin(values_to_drop)]
df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2440 entries, 0 to 2920
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subjectLabel    2440 non-null   object
 1   predicateLabel  2440 non-null   object
 2   objectLabel     2440 non-null   object
 3   subject         2440 non-null   object
 4   object          2440 non-null   object
 5   source          2440 non-null   object
dtypes: object(6)
memory usage: 133.4+ KB


,subjectLabel,predicateLabel,objectLabel,subject,object,source
0,pearl,has use,jewelry,Q43436,Q161439,Q83437
3,ruby,color,red,Q43088,Q3142,Q83437
6,pearl,subclass of,organic gem,Q43436,Q110688445,Q83437
7,pearl,subclass of,spherical body,Q43436,Q112511193,Q83437
8,pearl,subclass of,animal product,Q43436,Q629103,Q83437


In [4]:
import json
def spo2kg(df_spo_wiki):
    df = df_spo_wiki
    df_spo = df_spo_wiki[['subjectLabel','predicateLabel','objectLabel']]
    
    metadata = {}
    # Base Wikipedia URL for metadata
    wiki_base_url = "https://en.wikipedia.org/wiki/"
    
    for i in df.itertuples():
        # print(i.subjectLabel)
        
        # Add subject metadata (Wikidata ID and Wikipedia URL)
        if i.subjectLabel not in metadata:
            metadata[i.subjectLabel] = {
                "url": f"{wiki_base_url}{i.subjectLabel.replace(' ', '_')}",
                "wiki_id": i.subject
            }
        
        # Add object metadata (Wikidata ID and Wikipedia URL)
        if i.objectLabel not in metadata:
            metadata[i.objectLabel] = {
                "url": f"{wiki_base_url}{i.objectLabel.replace(' ', '_')}",
                "wiki_id": i.object
            }
    
    return df_spo, metadata

df_spo, metadata = spo2kg(df2)
df_spo.info()
df_spo.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2440 entries, 0 to 2920
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subjectLabel    2440 non-null   object
 1   predicateLabel  2440 non-null   object
 2   objectLabel     2440 non-null   object
dtypes: object(3)
memory usage: 76.2+ KB


,subjectLabel,predicateLabel,objectLabel
0,pearl,has use,jewelry
3,ruby,color,red
6,pearl,subclass of,organic gem
7,pearl,subclass of,spherical body
8,pearl,subclass of,animal product


In [5]:
csv_file= './data/3_data_drop.csv'
metadata_file='./data/3_metadata_drop.json'

df_spo.to_csv(csv_file, header=False, index=False)
with open(metadata_file, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2)

In [6]:
!kg add -f ./data/3_data_drop.csv
!kg meta -f ./data/3_metadata_drop.json
!kg start

🎉 Starting the app.
 * Serving Flask app 'kgsearch.app.app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Oct/2024 12:58:53] "GET /search/1/1/1/blue HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 12:58:56] "GET /search/1/1/1/blue;gemstone HTTP/1.1" 200 -
^C


In [7]:
df2

,subjectLabel,predicateLabel,objectLabel,subject,object,source
0,pearl,has use,jewelry,Q43436,Q161439,Q83437
3,ruby,color,red,Q43088,Q3142,Q83437
6,pearl,subclass of,organic gem,Q43436,Q110688445,Q83437
7,pearl,subclass of,spherical body,Q43436,Q112511193,Q83437
8,pearl,subclass of,animal product,Q43436,Q629103,Q83437
...,...,...,...,...,...,...
2915,garrelsite,crystal system,monoclinic crystal system,Q3758413,Q624543,Q429635
2916,gerstmannite,instance of,mineral species,Q3761316,Q12089225,Q429635
2917,gerstmannite,named after,Ewald Gerstmann,Q3761316,Q18343235,Q429635
2918,garrelsite,IMA status and/or rank,grandfathered mineral (G),Q3758413,Q13406846,Q429635


In [8]:
df3 = df2[['subject','object']].melt()[['value']].drop_duplicates()
df3.columns = ['wiki_id']
df3.info()
df3.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1372 entries, 0 to 4879
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   wiki_id  1372 non-null   object
dtypes: object(1)
memory usage: 21.4+ KB


,wiki_id
0,Q43436
1,Q43088
6,Q393356
8,Q5283
11,Q138979


In [9]:
df3.to_csv('./data/3_e.csv', index=False)